In [1]:
# baseado em https://stackabuse.com/k-nearest-neighbors-algorithm-in-python-and-scikit-learn/
# https://www.codigofluente.com.br/aula-06-scikit-learn-vizinhos-mais-proximos-knn/
# https://colab.research.google.com/github/littlecolumns/ds4j-notebooks/blob/master/classification/notebooks/Correcting%20for%20imbalanced%20datasets.ipynb#scrollTo=jq9376MGvTa7

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import cluster, neighbors, svm, metrics, preprocessing
np.set_printoptions(suppress=True)

In [3]:
originalDF = pd.read_csv('penguins_size.csv', encoding = "ISO-8859-1")

In [4]:
from sklearn import preprocessing
 
label_encoder = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
encodedDF = originalDF.copy()
encodedDF['island']= label_encoder.fit_transform(encodedDF['island'])
encodedDF['sex']= label_encoder.fit_transform(encodedDF['sex'])
encodedDF['species']= label_encoder.fit_transform(encodedDF['species'])

encodedDF = encodedDF[~np.isnan(encodedDF).any(axis=1)]
encodedDF.reset_index(drop=True, inplace=True)
encodedDF

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,0,2,39.1,18.7,181.0,3750.0,2
1,0,2,39.5,17.4,186.0,3800.0,1
2,0,2,40.3,18.0,195.0,3250.0,1
3,0,2,36.7,19.3,193.0,3450.0,1
4,0,2,39.3,20.6,190.0,3650.0,2
...,...,...,...,...,...,...,...
337,2,0,47.2,13.7,214.0,4925.0,1
338,2,0,46.8,14.3,215.0,4850.0,1
339,2,0,50.4,15.7,222.0,5750.0,2
340,2,0,45.2,14.8,212.0,5200.0,1


In [5]:
from sklearn.preprocessing import StandardScaler

scaledArr = StandardScaler().fit_transform(encodedDF.loc[:, ['island','culmen_length_mm','culmen_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex']].values)
scaledDF = pd.DataFrame(data = scaledArr, columns = ['island','culmen_length_mm','culmen_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex'])
scaledDF

,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,1.852870,-0.884499,0.785449,-1.418347,-0.564142,0.846836
1,1.852870,-0.811126,0.126188,-1.062250,-0.501703,-0.974660
2,1.852870,-0.664380,0.430462,-0.421277,-1.188532,-0.974660
3,1.852870,-1.324737,1.089724,-0.563715,-0.938776,-0.974660
4,1.852870,-0.847812,1.748985,-0.777373,-0.689020,0.846836
...,...,...,...,...,...,...
337,-0.914298,0.601305,-1.750171,0.931890,0.903175,-0.974660
338,-0.914298,0.527932,-1.445897,1.003109,0.809516,-0.974660
339,-0.914298,1.188289,-0.735923,1.501644,1.933419,0.846836
340,-0.914298,0.234440,-1.192335,0.789451,1.246590,-0.974660


In [6]:
scaledDF = pd.concat([scaledDF, encodedDF[['species']]], axis = 1)
scaledDF

,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex,species
0,1.852870,-0.884499,0.785449,-1.418347,-0.564142,0.846836,0
1,1.852870,-0.811126,0.126188,-1.062250,-0.501703,-0.974660,0
2,1.852870,-0.664380,0.430462,-0.421277,-1.188532,-0.974660,0
3,1.852870,-1.324737,1.089724,-0.563715,-0.938776,-0.974660,0
4,1.852870,-0.847812,1.748985,-0.777373,-0.689020,0.846836,0
...,...,...,...,...,...,...,...
337,-0.914298,0.601305,-1.750171,0.931890,0.903175,-0.974660,2
338,-0.914298,0.527932,-1.445897,1.003109,0.809516,-0.974660,2
339,-0.914298,1.188289,-0.735923,1.501644,1.933419,0.846836,2
340,-0.914298,0.234440,-1.192335,0.789451,1.246590,-0.974660,2


In [7]:
y = scaledDF.loc[:, ['species']].values
X = scaledDF.loc[:, ['island','culmen_length_mm','culmen_depth_mm', 'flipper_length_mm', 'body_mass_g', 'sex']].values
X

array([[ 1.85286967, -0.88449874,  0.78544923, -1.41834665, -0.56414208,
         0.84683555],
       [ 1.85286967, -0.81112573,  0.1261879 , -1.06225022, -0.50170305,
        -0.97465979],
       [ 1.85286967, -0.66437972,  0.43046236, -0.42127665, -1.18853234,
        -0.97465979],
       ...,
       [-0.91429814,  1.18828874, -0.73592307,  1.50164406,  1.93341896,
         0.84683555],
       [-0.91429814,  0.23443963, -1.19233476,  0.7894512 ,  1.24658968,
        -0.97465979],
       [-0.91429814,  1.09657248, -0.53307343,  0.86067049,  1.49634578,
         0.84683555]])

In [8]:
from imblearn.over_sampling import RandomOverSampler

OverResampler = RandomOverSampler()

X, y = OverResampler.fit_resample(X, y)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [10]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier(n_neighbors=1)
classifier.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

In [11]:
y_pred = classifier.predict(X_test)

In [12]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[34  0  0]
 [ 0 29  0]
 [ 0  0 28]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        34
           1       1.00      1.00      1.00        29
           2       1.00      1.00      1.00        28

    accuracy                           1.00        91
   macro avg       1.00      1.00      1.00        91
weighted avg       1.00      1.00      1.00        91

